In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf
import json
import time
from requests.auth import HTTPBasicAuth
from aadatapipelinecore.core.utils.http import plain_get


# source_path='s3://b2c-prod-data-pipeline-unified-review/unified/review.v1/fact/process_granularity=monthly/'
source_path='s3://b2c-prod-data-pipeline-unified-review/unified/review_bulk_json.v3/'



s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-review'))
path_list = s3_bucket_list.all(prefix="unified/review_bulk_json.v3/", depth_is_1=True)
path_list = path_list[108:-1]

# print path_list
# print len(path_list)

# index_list=[]



for path in path_list[:2]:
    print path
    total=spark.read.json("s3://b2c-prod-data-pipeline-unified-review/{}".format(path)).filter("_identifier is not null").count()

    new_path=path.split("=")[1][:7].replace("-","")
    
    index_list.append(("int-ss-review_v1-apple-store-ios-all-{}".format(new_path),"int-ss-review_v1-google-play-android-all-{}".format(new_path) ,total))



def es_doc(index):
    print index[0], index[1], index[2]
    headers = {
        'content-type': 'application/json'
    }
    auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    ios_url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_count".format(index[0])
    # print ios_url
    ios_resp = plain_get(ios_url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    ios_doc = json.loads(ios_resp.text)
    print ios_doc["count"]
    gp_url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_count".format(index[1])
    gp_resp = plain_get(gp_url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
    gp_doc = json.loads(gp_resp.text)
    print gp_doc
    print gp_doc["count"]
    total_count = int(ios_doc["count"]) + int(gp_doc["count"])
    if total_count != index[2] :
        print 'not equal! path : {} unified count: {} , db count: {}'.format(index[0], index[2] ,total_count)

print index_list
es_doc(index_list[3])
    # source = doc['_source']
    # source['review_id'] = doc["_id"]
    # return source


# for path in path_list[150:]:
#     print path.split("/")[4].split("=")[1]

#     ("int-ss-review_v1-apple-store-ios-all-201901", ios_samples_df), 
#     ("int-ss-review_v1-google-play-android-all-201901", android_samples_df)]:

# def compare(doc, row):
#     keys = [u"review_id", u'_identifier', u'app_id', u'chinese_content', u'chinese_reply', u'chinese_title', u'content', u'content_language', u'country_code', u'date', u'device_code', u'english_content', u'english_reply', u'english_title', u'japanese_content', u'japanese_reply', u'japanese_title', u'korean_content', u'korean_reply', u'korean_title', u'language', u'market_code', u'other_language_content', u'other_language_reply', u'other_language_title', u'product_version', u'rating', u'reply', u'reply_date', u'reply_language', u'title', u'title_language', u'user_device', u'user_id', u'user_language', u'user_name', u'user_purchased']
#     row_dict =  row.asDict()
#     for key in keys:
#         if key in ("app_id", "_identifier"):
#             assert str(doc[key]) == str(row_dict[key]), "{} is diffrent".format(key)
#         else:
#             assert doc[key] == row_dict[key], "{} is diffrent".format(key)
# def es_doc(index, doc_id, routing_id):
#     headers = {
#         'content-type': 'application/json'
#     }
#     auth = HTTPBasicAuth("bdp","C38vEJEuraCw")
    
#     url = "http://internal-internel-review-es-v7-1286999769.us-east-1.elb.amazonaws.com:19200/{}/_doc/{}?routing={}".format(index, doc_id, routing_id)
#     resp = plain_get(url, timeout=30, auth=auth, headers=headers, is_resource_of_appannie=False)
#     doc = json.loads(resp.text)
#     source = doc['_source']
#     source['review_id'] = doc["_id"]
#     return source
# for idx, rows in samples:
#     for row in rows:
#         doc = es_doc(idx, row.review_id, row.app_id)
#         compare(doc, row)


